In [8]:
import pandas as pd
from shutil import copyfile, copy
from tqdm.notebook import tqdm
from google.colab.patches import cv2_imshow
import cv2
import os
import glob
import random
import numpy as np
from PIL import Image  # Python Image Library - Image Processing

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Jaffe

In [3]:
# https://stackoverflow.com/questions/123198/how-can-a-file-be-copied

jaffe = pd.read_csv('drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/JAFFE/jaffe_final.csv')
jaffe.columns = ['happy', 'sad', 'surprise', 'angry', 'disgust', 'fear', 'pic','path', 'label']
dict_jaffe = {0:'happy', 1:'sad', 2:'surprise', 3:'angry', 4:'disgust', 5:'fear'}
jaffe['label'] = jaffe['label'].replace(dict_jaffe) 
jaffe.head(2)

,happy,sad,surprise,angry,disgust,fear,pic,path,label
0,2.87,2.52,2.10,1.97,1.97,2.06,KM.NE1,drive/MyDrive/AIHealthcare/AIcare_Phrase1/data...,happy
1,2.87,2.42,1.58,1.84,1.77,1.77,KM.NE2,drive/MyDrive/AIHealthcare/AIcare_Phrase1/data...,happy


In [4]:
for i in tqdm(range(jaffe.shape[0])):
  emotion = jaffe['label'][i] 
  img_path = jaffe['path'][i] 
  img_name = img_path[64:]
  terminal_path = "drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/ck_kaggle_jaffe/{0}/{1}".format(emotion, img_name)
  copyfile(img_path, terminal_path)

**change tiff -> png**

In [10]:
folders = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise']
for folder in folders:
  for file in glob.glob("drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/ck_kaggle_jaffe/"+folder+"/*.tiff"):
    im = Image.open(file)
    rgb_im = im.convert('RGB')
    rgb_im.save(file.replace("tiff", "png"))

In [13]:
for folder in folders:
  folder_path = ('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/ck_kaggle_jaffe/'+folder+'/')
  test = os.listdir(folder_path)
  for images in tqdm(test):
      if images.endswith(".tif"):
        os.remove(os.path.join(folder_path, images))
      elif images.endswith(".tiff"):
        os.remove(os.path.join(folder_path, images))        

# kaggle

In [14]:
def prepare_data(data):    
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, 'pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label

def row2image(row):
    pixels, emotion = row['pixels'], emotion_map[row['emotion']]
    img = np.array(pixels.split())
    img = img.reshape(48,48)
    image = np.zeros((48,48,3))
    image[:,:,0] = img
    image[:,:,1] = img
    image[:,:,2] = img
    return np.array([image.astype(np.uint8), emotion])

In [16]:
kaggle = pd.read_csv('drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/kaggle/fer2013/fer2013.csv')
emotions = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'sad', 5: 'surprise', 6: 'neutral'}

image_ar, image_label = prepare_data(kaggle)
print(image_ar.shape)
print(image_label.shape)
image_label = pd.DataFrame(image_label, columns=['label']).replace(emotions)

for i in tqdm(range(kaggle.shape[0])):
  filename = 'kaggle.png'
  rgb = cv2.resize(image_ar[i], (224, 224), interpolation = cv2.INTER_CUBIC)
  # rgb = cv2.cvtColor(image_ar[i].astype(np.float32),cv2.COLOR_GRAY2RGB)
  cv2.imwrite(filename, rgb)
  dst = 'kaggle_'+str(i)+'.png'
  terminal_path = "drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/ck_kaggle_jaffe/{0}/{1}".format(image_label['label'][i], dst)
  copyfile('/content/kaggle.png', terminal_path)

(35887, 48, 48)
(35887,)


# CK+

In [17]:
random.seed(123)
emotions = ["neutral", "angry", "contempt", "disgust", "fear", "happy", "sad", "surprise"]

In [ ]:
%cd drive/MyDrive/AIHealthcare/FER_CK

In [19]:
observations = glob.glob("source_emotion/*")

In [ ]:
neutral_file_dict={}
for obs in tqdm(observations):
    obs_id = str(obs[-4:])
    neutral_file_dict[obs_id]=list()
    emotions_folders = "{0}/*".format(str(obs))
    emotions_sessions =  glob.glob(emotions_folders)
    for each_emotion_session in emotions_sessions:
        emotion_sequence_folder = "{0}/*".format(each_emotion_session)
        emotion_sequence_files = glob.glob(emotion_sequence_folder)
        for emotion_output in emotion_sequence_files:
            emotion_seq_no = emotion_output[20:23]
            file = open(emotion_output, 'r')
            emotion= int(float(file.readline()))
            peak_frame_emotion = sorted(glob.glob("source_images/{0}/{1}/*".format(obs_id,emotion_seq_no)))[-1]
            first_frame_emotion = sorted(glob.glob("source_images/{0}/{1}/*".format(obs_id,emotion_seq_no)))[0]
            neutral_file_dict[obs_id].append(first_frame_emotion)
            peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/ck_kaggle_jaffe/{0}/{1}".format(emotions[emotion], peak_frame_emotion[23:])
            copyfile(peak_frame_emotion, peak_emotion_img_name)
%cd ../../../../

In [ ]:
%cd drive/MyDrive/AIHealthcare/FER_CK
for key in neutral_file_dict.keys():
    if len(neutral_file_dict[key])>0:
        # randomly copy neutral frames to neutral folder
        source_file = random.choice(neutral_file_dict[key])
        neutral_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/ck_kaggle_jaffe/neutral/{0}".format(source_file[23:])
        copyfile(source_file, neutral_emotion_img_name)
%cd ../../../../

In [ ]:
ls '/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/ck_kaggle_jaffe/neutral'

In [28]:
img = cv2.imread('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/ck_kaggle_jaffe/neutral/S999_001_00000001.png')
cv2_imshow(img)

# FERG_DB256

### aia

In [ ]:
ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/aia/aia_anger/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[96:101].replace('anger', 'angry'), filepath[102:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/aia/aia_disgust/*.png'))
for filepath in tqdm(ans):
  img = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[96:103], filepath[104:])
  copyfile(filepath, img)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/aia/aia_fear/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[96:100].replace('anger', 'angry'), filepath[101:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/aia/aia_joy/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[96:99], filepath[100:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/aia/aia_neutral/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[96:103], filepath[104:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/aia/aia_sadness/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[96:99], filepath[104:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/aia/aia_surprise/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[96:104], filepath[105:])
  copyfile(filepath, peak_emotion_img_name)

### bonnie

In [ ]:
ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/bonnie/bonnie_anger/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[102:107].replace('anger', 'angry'), filepath[111:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/bonnie/bonnie_disgust/*.png'))
for filepath in tqdm(ans):
  img = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[102:109], filepath[110:])
  copyfile(filepath, img)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/bonnie/bonnie_fear/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[102:106], filepath[107:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/bonnie/bonnie_joy/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[102:105], filepath[106:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/bonnie/bonnie_neutral/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[102:109], filepath[110:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/bonnie/bonnie_sadness/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[102:105], filepath[110:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/bonnie/bonnie_surprise/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[102:110], filepath[111:])
  copyfile(filepath, peak_emotion_img_name)

### jules

In [ ]:
ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/jules/jules_anger/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[100:105].replace('anger', 'angry'), filepath[106:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/jules/jules_disgust/*.png'))
for filepath in tqdm(ans):
  img = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[100:107], filepath[108:])
  copyfile(filepath, img)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/jules/jules_fear/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[100:104], filepath[105:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/jules/jules_joy/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[100:103], filepath[104:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/jules/jules_neutral/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[100:107], filepath[108:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/jules/jules_sadness/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[100:103], filepath[108:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/jules/jules_surprise/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[100:108], filepath[109:])
  copyfile(filepath, peak_emotion_img_name)

### mery

In [ ]:
ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/mery/mery_anger/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[98:98+5].replace('anger', 'angry'), filepath[98+5+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/mery/mery_disgust/*.png'))
for filepath in tqdm(ans):
  img = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[98:98+7], filepath[98+7+1:])
  copyfile(filepath, img)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/mery/mery_fear/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[98:98+4], filepath[98+4+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/mery/mery_joy/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[98:98+3], filepath[98+3+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/mery/mery_neutral/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[98:98+7], filepath[98+7+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/mery/mery_sadness/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[98:98+3], filepath[98+7+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/mery/mery_surprise/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[98:98+7], filepath[98+7+1:])
  copyfile(filepath, peak_emotion_img_name)

### ray

In [ ]:
ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/ray/ray_anger/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[96:96+5].replace('anger', 'angry'), filepath[96+5+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/ray/ray_disgust/*.png'))
for filepath in tqdm(ans):
  img = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[96:96+7], filepath[96+7+1:])
  copyfile(filepath, img)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/ray/ray_fear/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[96:96+4], filepath[96+4+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/ray/ray_joy/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[96:96+3], filepath[96+3+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/ray/ray_neutral/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[96:96+7], filepath[96+7+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/ray/ray_sadness/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[96:96+3], filepath[96+8:])
#   copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/ray/ray_surprise/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[96:96+8], filepath[96+7+1:])
  copyfile(filepath, peak_emotion_img_name)

### malcolm

In [ ]:
ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/malcolm/malcolm_anger/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[104:104+5].replace('anger', 'angry'), filepath[104+5+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/malcolm/malcolm_disgust/*.png'))
for filepath in tqdm(ans):
  img = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[104:104+7], filepath[104+7+1:])
  copyfile(filepath, img)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/malcolm/malcolm_fear/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[104:104+4], filepath[104+4+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/malcolm/malcolm_joy/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[104:104+3], filepath[104+3+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/malcolm/malcolm_neutral/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[104:104+7], filepath[104+7+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/malcolm/malcolm_sadness/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[104:104+3], filepath[104+8:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB_256/FERG_DB_256/malcolm/malcolm_surprise/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(filepath[104:104+8], filepath[104+7+1:])
  copyfile(filepath, peak_emotion_img_name)

# FERG

### aia

In [ ]:
ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/aia/aia_anger/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[93:93+5].replace('anger', 'angry'), 'FERG_DB' + filepath[93+5+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/aia/aia_disgust/*.png'))
for filepath in tqdm(ans):
  img = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[93:93+7], 'FERG_DB'+filepath[93+7+1:])
  copyfile(filepath, img)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/aia/aia_fear/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[93:93+4], 'FERG_DB'+filepath[93+4+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/aia/aia_joy/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[93:93+3], 'FERG_DB'+filepath[93+3+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/aia/aia_neutral/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[93:93+7], 'FERG_DB'+filepath[93+7+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/aia/aia_sadness/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[93:93+3], 'FERG_DB'+filepath[93+3+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/aia/aia_surprise/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[93:93+7], 'FERG_DB'+filepath[93+7+1:])
  copyfile(filepath, peak_emotion_img_name)

### bonnie

In [ ]:
ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/bonnie/bonnie_anger/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[99:99+5].replace('anger', 'angry'), 'FERG_DB'+filepath[99+5+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/bonnie/bonnie_disgust/*.png'))
for filepath in tqdm(ans):
  img = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[99:99+7], 'FERG_DB'+filepath[99+7+1:])
  copyfile(filepath, img)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/bonnie/bonnie_fear/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[99:99+4], 'FERG_DB'+filepath[99+4+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/bonnie/bonnie_joy/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[99:99+3], 'FERG_DB'+filepath[99+3+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/bonnie/bonnie_neutral/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[99:99+7], 'FERG_DB'+filepath[99+7+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/bonnie/bonnie_sadness/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[99:99+3], 'FERG_DB'+filepath[99+8:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/bonnie/bonnie_surprise/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[99:99+7], 'FERG_DB'+filepath[99+7+1:])
  copyfile(filepath, peak_emotion_img_name)

### jules

In [ ]:
ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/jules/jules_anger/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[97:97+5].replace('anger', 'angry'), 'FERG_DB'+filepath[97+5+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/jules/jules_disgust/*.png'))
for filepath in tqdm(ans):
  img = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[97:97+7], 'FERG_DB'+filepath[97+7+1:])
  copyfile(filepath, img)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/jules/jules_fear/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[97:97+4], 'FERG_DB'+filepath[97+4+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/jules/jules_joy/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[97:97+3], 'FERG_DB'+filepath[97+3+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/jules/jules_neutral/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[97:97+7], 'FERG_DB'+filepath[97+7+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/jules/jules_sadness/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[97:97+3], 'FERG_DB'+filepath[97+8:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/jules/jules_surprise/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[97:97+8], 'FERG_DB'+filepath[97+8+1:])
  copyfile(filepath, peak_emotion_img_name)

### malcolm

In [ ]:
ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/malcolm/malcolm_anger/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[101:101+5].replace('anger', 'angry'), 'FERG_DB'+filepath[101+5+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/malcolm/malcolm_disgust/*.png'))
for filepath in tqdm(ans):
  img = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[101:101+7], 'FERG_DB'+filepath[101+7+1:])
  copyfile(filepath, img)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/malcolm/malcolm_fear/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[101:101+4], 'FERG_DB'+filepath[101+4+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/malcolm/malcolm_joy/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[101:101+3], 'FERG_DB'+filepath[101+3+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/malcolm/malcolm_neutral/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[101:101+7], 'FERG_DB'+filepath[101+7+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/malcolm/malcolm_sadness/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[101:101+3], 'FERG_DB'+filepath[101+8:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/malcolm/malcolm_surprise/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[101:101+8], 'FERG_DB'+filepath[101+8+1:])
  copyfile(filepath, peak_emotion_img_name)

### mery

In [ ]:
ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/mery/mery_anger/*.png'))
ans[0][95:]

'anger/mery_anger_1.png'

In [ ]:
ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/mery/mery_anger/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[95:95+5].replace('anger', 'angry'), 'FERG_DB'+filepath[95+5+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/mery/mery_disgust/*.png'))
for filepath in tqdm(ans):
  img = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[95:95+7], 'FERG_DB'+filepath[95+7+1:])
  copyfile(filepath, img)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/mery/mery_fear/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[95:95+4], 'FERG_DB'+filepath[95+4+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/mery/mery_joy/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[95:95+3], 'FERG_DB'+filepath[95+3+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/mery/mery_neutral/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[95:95+7], 'FERG_DB'+filepath[95+7+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/mery/mery_sadness/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[95:95+3], 'FERG_DB'+filepath[95+8:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/mery/mery_surprise/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[95:95+8], 'FERG_DB'+filepath[95+8+1:])
  copyfile(filepath, peak_emotion_img_name)

### ray

In [ ]:
ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/ray/ray_anger/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[93:93+5].replace('anger', 'angry'), 'FERG_DB'+filepath[93+5+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/ray/ray_disgust/*.png'))
for filepath in tqdm(ans):
  img = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[93:93+7], 'FERG_DB'+filepath[93+7+1:])
  copyfile(filepath, img)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/ray/ray_fear/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[93:93+4], 'FERG_DB'+filepath[93+4+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/ray/ray_joy/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[93:93+3], 'FERG_DB'+filepath[93+3+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/ray/ray_neutral/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[93:93+7], 'FERG_DB'+filepath[93+7+1:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/ray/ray_sadness/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[93:93+3], 'FERG_DB'+filepath[93+8:])
  copyfile(filepath, peak_emotion_img_name)

ans = sorted(glob.glob('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/FERG_DB/FERG_DB/FERG_DB_orig/ray/ray_surprise/*.png'))
for filepath in tqdm(ans):
  peak_emotion_img_name = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB/{0}/{1}".format(filepath[93:93+8], 'FERG_DB'+filepath[93+8+1:])
  copyfile(filepath, peak_emotion_img_name)

# FacesDB
http://app.visgraf.impa.br/database/faces/

In [ ]:
# ls '/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/facesdb'

In [ ]:
for i in tqdm(ans):
  b = sorted(glob.glob(i+'/tif/*.tif'))
  for k in b: # for k in tqdm(b):
    emotion = k[78:80]
    if emotion == '00':
      A = 'neutral'
    elif emotion == '01':
      A = 'joy'
    elif emotion == '02':
      A = 'sad'
    elif emotion == '03':
      A = 'surprise'
    elif emotion == '04':   
      A = 'angry'
    elif emotion == '05':
      A = 'disgust'
    elif emotion == '06':
      A = 'fear'
    img = "/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset_separate/FERG_DB_256/{0}/{1}".format(A, k[73:])
    copyfile(k, img)

In [ ]:
ls '/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset/angry'

In [ ]:
# import os
# os.remove(file) for file in os.listdir('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset/angry/') if file.endswith('.tif')
# os.remove(file) for file in os.listdir('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset/angry/') if file.endswith('.tiff')

In [ ]:
import os
folder_path = ('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset/surprise/')
test = os.listdir(folder_path)
for images in tqdm(test):
    if images.endswith(".tif"):
        os.remove(os.path.join(folder_path, images))

folder_path = ('/content/drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/trainset/surprise/')
test = os.listdir(folder_path)
for images in tqdm(test):
    if images.endswith(".tiff"):
        os.remove(os.path.join(folder_path, images))